In [ ]:
# set Keras backend 
import os 
os.environ["KERAS_BACKEND"] = "torch"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import random

# set warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

# import modules and components
from FEXT.commons.utils.dataloader.generators import ML_model_dataloader
from FEXT.commons.utils.dataloader.serializer import DataSerializer, ModelSerializer
from FEXT.commons.utils.learning.inference import ImageEncoding
from FEXT.commons.utils.validation.images import ImageReconstruction
from FEXT.commons.utils.validation.checkpoints import ModelEvaluationSummary
from FEXT.commons.utils.validation.reports import evaluation_report

## Generate evaluation summary for all checkpoints

---

Generate a summary of all checkpoints and their hyperparameters

In [2]:
summarizer = ModelEvaluationSummary()
checkpoints_summary = summarizer.checkpoints_summary()

## Load data and model

---

Load the model from available checkpoints. If only a single checkpoint is available, it will be automatically loaded. Once the full model is deserialized, the encoder component will be isolated to process images and evaluate the reconstruction error.


In [ ]:
# selected and load the pretrained model, then print the summary 
modelserializer = ModelSerializer()         
model, configuration, history, checkpoint_path = modelserializer.select_and_load_checkpoint()
model.summary(expand_nested=True)

# isolate the encoder from the autoencoder model, and use it for inference     
encoder = ImageEncoding(model, configuration)
encoder_model = encoder.encoder_model

In [4]:
evaluation_batch_size = 30
num_images_to_evaluate = 10

In [5]:
dataserializer = DataSerializer(configuration)
train_data, validation_data = dataserializer.load_preprocessed_data(checkpoint_path)
train_images_batch = [dataserializer.load_image(path) for path in random.sample(train_data, num_images_to_evaluate)]
validation_images_batch = [dataserializer.load_image(path) for path in random.sample(validation_data, num_images_to_evaluate)]

# Model performance evaluation

---


### Evaluation of loss and metrics

We use huber loss for the Convolutional Auto Encoder (CAE) and cosine similarity as a metric to evaluate similarity between input and reconstructed output. 

$$
\text{Cosine Similarity} = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}
$$

A cosine similarity value closer to **1** indicates that the original and reconstructed images are highly simil

In [ ]:
train_dataset, validation_dataset = ML_model_dataloader(train_data, validation_data, configuration, evaluation_batch_size)
evaluation_report(model, train_dataset, validation_dataset) 

### Reconstruction evaluation

Compare reconstructed images to original pictures to qualitatively evaluate the performance of the FeXT autoencoder model.

In [ ]:
validator = ImageReconstruction(model)
print('Visualize images recontructed from train dataset')
validator.visualize_reconstructed_images(train_images_batch)

In [ ]:
print('Visualize images recontructed from validation dataset')
validator.visualize_reconstructed_images(validation_images_batch)

Visualize the original image and the reconstructed images, together with the corresponding features vector. The raw vector has shape 4x4x512 and is reshaped to be 64x128

In [ ]:
train_dataset, validation_dataset = ML_model_dataloader(train_data, validation_data, configuration, batch_size=1)
 
validator.visualize_latent_space(model, train_dataset, checkpoint_path, method='PCA', 
                                 n_components=3, num_images=num_images_to_evaluate)